# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[35, 32]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,0,0.563,6,Lordshenge,35:32,Lionwish,0,2,town,0,1,35,32
NArea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,void,NaN,NaN,NaN,NaN
SArea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,void,NaN,NaN,NaN,NaN
EArea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,void,NaN,NaN,NaN,NaN
WArea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,void,NaN,NaN,NaN,NaN


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

True

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
28:3,3,0.0,28,28:3,6.0,desert,NaN,4,Puremain,0,1,0,0.703
24:4,4,1.0,24,24:4,6.0,desert,NaN,4,Puremain,0,1,0,0.720
29:4,4,0.0,29,29:4,6.0,desert,NaN,3,Earthvale,0,1,0,0.759
28:5,5,0.0,28,28:5,6.0,desert,NaN,3,Earthvale,0,1,0,0.850
29:6,6,1.0,29,29:6,6.0,desert,NaN,3,Earthvale,0,1,0,0.683


You can also _mask_ the terrain where the character has no knowledge. 

In [11]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,1.0,0,0:0,0.0,ocean,NaN,4,Puremain,0,1,0,0.515
1:0,0,1.0,1,1:0,0.0,ocean,NaN,4,Puremain,0,1,0,0.132
2:0,0,1.0,2,2:0,0.0,ocean,NaN,4,Puremain,0,1,0,0.473
3:0,0,1.0,3,3:0,0.0,ocean,NaN,4,Puremain,0,1,0,0.507
4:0,0,1.0,4,4:0,0.0,ocean,NaN,4,Puremain,0,1,0,0.415


## Analysis via list comprehension

Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [8]:
[[person.name for person in t.population] for t in world.towns]

[['Erffale', 'Alicarrunt', 'Joshim', 'Erklaf', 'Chissash'],
 ['Newillut', 'Atendolen', 'Krasassec', 'Tonsust', 'Olfaddunder'],
 ['Fiharkadle', 'Olkrackust', 'Dowallomir'],
 ['Thagennnan', 'Leboldent', 'Xirushaker', 'Hannak', 'Anapprion'],
 ['Drayellep', 'Xangnan', 'Lezennnan'],
 ['Thentoh', 'Mezodin', 'Quamendodin', 'Locarrak', 'Trimsar'],
 ['Ekihorkar', 'Ekikkin'],
 ['Dondir'],
 ['Tatendef', 'Stelordat', 'Chiladdark'],
 ['Pekknan', 'Hatallad', 'Secksow'],
 ['Pelandumber', 'Fewillak'],
 ['Eroven', 'Thaladdech'],
 ['Vofaddor'],
 ['Groharkit'],
 ['Drakkand'],
 ['Moflomir']]

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [12]:
[[(person.name,person.temperment) for person in t.population if person.temperment>.5] for t in world.towns]

[[('Erffale', 0.85), ('Chissash', 0.83)],
 [('Atendolen', 0.72), ('Tonsust', 1.0)],
 [('Olkrackust', 0.61)],
 [('Leboldent', 0.91), ('Xirushaker', 0.59), ('Anapprion', 0.63)],
 [],
 [],
 [('Ekihorkar', 0.85)],
 [('Dondir', 0.54)],
 [('Stelordat', 0.94)],
 [('Secksow', 0.86)],
 [],
 [('Thaladdech', 0.56)],
 [],
 [],
 [('Drakkand', 0.52)],
 []]

You can also get summary stats on towns. For example, the average temperment per town.

In [41]:
[(t.name,np.mean([person.temperment for person in t.population])) for t in world.towns]

[('Artblade', 0.40599999999999997),
 ('Fairspring', 0.48999999999999994),
 ('Oakspring', 0.36000000000000004),
 ('Hardforth', 0.5359999999999999),
 ('Heircatch', 0.32666666666666666),
 ('Ravencrawl', 0.322),
 ('Fenforth', 0.495),
 ('Starpike', 0.54),
 ('Tearrest', 0.33),
 ('Gleamcombe', 0.48333333333333334),
 ('Badgerblade', 0.21000000000000002),
 ('Badgerchild', 0.48000000000000004),
 ('Lordshenge', 0.12),
 ('Lionglen', 0.36),
 ('Earthborourgh', 0.52),
 ('Ratbrook', 0.49)]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [47]:
w.get_people_where_char_has_visited(world)

[{'town': town of Lordshenge: population: 1 location: [35,32] founded 8,
  'people': [Vofaddor the Speaker of Lordshenge]}]

In [50]:
r = w.get_people_where_char_has_visited(world)[0]
r

{'town': town of Lordshenge: population: 1 location: [35,32] founded 8,
 'people': [Vofaddor the Speaker of Lordshenge]}

In [55]:
r['town'].nation

'Lordshenge'

In [40]:
w.get_relationships_node_map(world)

KeyError: 'nation'

In [16]:
where_the_char_has_been['nation'].dropna()

key
35:32    Lionwish
Name: nation, dtype: object

In [19]:
nations_where_character_has_been = np.unique(where_the_char_has_been['nation'].dropna())
nations_where_character_has_been

array(['Lionwish'], dtype=object)

You can use this to get lists of people who the character knows. 

In [31]:
[[t for t in n.get_all_towns(world)] for n in world.nations if n.name in nations_where_character_has_been]

[[capitol of Gleamcombe: population: 3 location: [36,26] founded 5,
  town of Lordshenge: population: 1 location: [35,32] founded 8]]

In [32]:
[[t for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

[]

In [35]:
[(T.nation,T.name) for T in world.towns]

[('Artblade', 'Artblade'),
 ('Fairspring', 'Fairspring'),
 ('Oakspring', 'Oakspring'),
 ('Hardforth', 'Hardforth'),
 ('Heircatch', 'Heircatch'),
 ('Ravencrawl', 'Ravencrawl'),
 ('Fenforth', 'Fenforth'),
 ('Starpike', 'Starpike'),
 ('Tearrest', 'Tearrest'),
 ('Gleamcombe', 'Gleamcombe'),
 ('Badgerblade', 'Badgerblade'),
 ('Badgerchild', 'Badgerchild'),
 ('Lordshenge', 'Lordshenge'),
 ('Lionglen', 'Lionglen'),
 ('Earthborourgh', 'Earthborourgh'),
 ('Ratbrook', 'Ratbrook')]

In [36]:
[[t.get_person_data() for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

[]

In [37]:
masked_world.loc[world.Character.get_location_key()]

y                            32
rainfall                      2
x                            35
key                       35:32
elevation                     6
terrain                    town
feature              Lordshenge
nation number                 0
nation                 Lionwish
visited                       1
aware                         0
turn_last_visited             0
danger                    0.563
Name: 35:32, dtype: object

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [38]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
35:32,32,2.0,35,35:32,6.0,town,Lordshenge,0,Lionwish,1,0,0,0.563


## Getting the diplomacy and relationships data for the journal page

In [19]:
rel = w.get_people_where_char_has_visited(world)
rel

[{'town': town of Earthmark: population: 2 location: [18,17] founded 6,
  'people': [Shensith the Speaker of Earthmark, Vogennart the commoner]}]

In [20]:
w.get_relationships_node_map(world)

[{'name': 'Earthmark',
  'title': 'town of Earthmark',
  'nation': 'Brinehaven',
  'population': 2,
  'type': 'town',
  'location': '18:17',
  'founded year': 6,
  'national fealty': 1,
  'children': [{'name': 'Shensith',
    'role': 'Speaker of Earthmark',
    'temperment': 0.43,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Vogennart',
    'role': 'commoner',
    'temperment': 0.25,
    'loyalty': 0.5,
    'type': 'person'}]}]

In [21]:
hierarchy = [{'nation':i.name,
              'children':i.diplomacy} for i in world.nations]



In [22]:
p = rel[0]['town'].population[0]


In [26]:
def get_people_where_char_has_visited(world):
    """
    returns a dictionary of {{"town":[<obj>],"people":[<obj>]}
    filtered to where the character has been. 
    """
    where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                    (world.df_features['terrain']=='town')].dropna()
    towns_and_people = [{"town":T,
                         "people":[t for t in T.population]} 
                        for T in world.towns if T.name in np.unique(w.where_the_char_has_been['feature'])]
    return towns_and_people

def get_relationships_node_map(world):
    nodes = [{'name':r['town'].name,
              'title':str(r['town']).split(":")[0],
              'nation':r['town'].diplomacy['nation'],
              'population':r['town'].pop,
              'type':r['town'].type,
              'location':r['town'].key,
              'founded year':r['town'].founded,
              'national fealty':r['town'].diplomacy['national fealty'],
             'children': [{'name':p.name,
                          'role':p.role,
                          'temperment':p.temperment,
                          'loyalty':p.loyalty,
                          'type':'person'} for p in r['people']]} 
             for r in w.get_people_where_char_has_visited(world)]
    return nodes
        
w.get_people_where_char_has_visited(world)

[{'town': town of Earthmark: population: 2 location: [18,17] founded 6,
  'people': [Shensith the Speaker of Earthmark, Vogennart the commoner]}]

In [28]:
relationships = w.get_relationships_node_map(world)
relationships

[{'name': 'Earthmark',
  'title': 'town of Earthmark',
  'nation': 'Brinehaven',
  'population': 2,
  'type': 'town',
  'location': '18:17',
  'founded year': 6,
  'national fealty': 1,
  'children': [{'name': 'Shensith',
    'role': 'Speaker of Earthmark',
    'temperment': 0.43,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Vogennart',
    'role': 'commoner',
    'temperment': 0.25,
    'loyalty': 0.5,
    'type': 'person'}]}]

In [36]:
nations = [s['nation'] for s in relationships]
nations

['Brinehaven']

In [40]:
[i for i in relationships]

[{'name': 'Earthmark',
  'title': 'town of Earthmark',
  'nation': 'Brinehaven',
  'population': 2,
  'type': 'town',
  'location': '18:17',
  'founded year': 6,
  'national fealty': 1,
  'children': [{'name': 'Shensith',
    'role': 'Speaker of Earthmark',
    'temperment': 0.43,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Vogennart',
    'role': 'commoner',
    'temperment': 0.25,
    'loyalty': 0.5,
    'type': 'person'}]}]

In [49]:
[{'nation':i,'towns':[t for t in relationships if t['nation']==i]} for i in nations]

[{'nation': 'Brinehaven',
  'towns': [{'name': 'Earthmark',
    'title': 'town of Earthmark',
    'nation': 'Brinehaven',
    'population': 2,
    'type': 'town',
    'location': '18:17',
    'founded year': 6,
    'national fealty': 1,
    'children': [{'name': 'Shensith',
      'role': 'Speaker of Earthmark',
      'temperment': 0.43,
      'loyalty': 0.5,
      'type': 'person'},
     {'name': 'Vogennart',
      'role': 'commoner',
      'temperment': 0.25,
      'loyalty': 0.5,
      'type': 'person'}]}]}]

In [46]:
[i for i in nations]

['Brinehaven']